# Imports

In [172]:
from botiverse import chat_gui
from botiverse.bots import DeepTODS
from botiverse.models import BERTConfig, TRIPPYConfig
import gdown
import os

# Download necessary files to run the example

In [173]:
# Download ontology
f_id = '1rPmf2RFT5BiFOrc96zqAjZiBUyZEjxbu'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'ontology.json'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Ontology downloaded successfully.')
else:
    print('Ontology already exists. Skipping download.')

Ontology already exists. Skipping download.


In [174]:
# Download label maps
f_id = '1PBKA6PyMDyQMYqmMycMb2rSeU1RHY6OF'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'label_maps.json'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Label maps downloaded successfully.')
else:
    print('Label maps already exists. Skipping download.')

Label maps already exists. Skipping download.


In [175]:
# Download DST weights trained on MultiWOZ
f_id = '1P88oxRs3ZolIS66nCXOyW9QmlOI32dz9'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'model.pt'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Model downloaded successfully.')
else:
    print('Model already exists. Skipping download.')

Model already exists. Skipping download.


# Const

In [176]:
CHATBOT_NAME = 'Tody'

DOMAINS = ["hotel", "train", "restaurant", "attraction", "taxi"]

ONTOLOGY_PATH = './ontology.json'

LABEL_MAPS_PATH = './label_maps.json'

POLICY = 'Priority' # Priority or Random

START = [
    {
        'utterance': 'Hi I am Tody, I can help you reserve a hotel or a taxi?',
        'slots': [],
        'system_act': {}
    }
]

TEMPLATES = [
    {
        'utterance': 'Do you have a price reference?',
        'slots': ['hotel-pricerange'],
        'system_act': {}
    },
    {
        'utterance': 'What day would you like to stay and how many days?',
        'slots': ['hotel-book_day', 'hotel-book_stay'],
        'system_act': {}
    },
    {
        'utterance': 'Which area of town do you prefer to stay in?',
        'slots': ['hotel-area'],
        'system_act': {}
    },
    {
        'utterance': 'I have one called the continental hotel.',
        'slots': ['hotel-name'],
        'system_act': {'hotel-name': ['continental hotel']}
    },
    {
        'utterance': 'Do you need parking or internet?',
        'slots': ['hotel-parking', 'hotel-internet'],
        'system_act': {}
    },
    {
        'utterance': 'How many people is the booking for?',
        'slots': ['hotel-book_people'],
        'system_act': {}
    },
    
    {
        'utterance': 'What is your desired destination?',
        'slots': ['taxi-destination'],
        'system_act': {}
    },
    {
        'utterance': 'From where you will be departing, and what time do you want to leave?',
        'slots': ['taxi-departure', 'taxi-leaveAt'],
        'system_act': {}
    }
]

NON_REFERABLE_SLOTS = ['hotel-stars', 'hotel-internet', 'hotel-parking']

NON_REFERABLE_PAIRS = [('hotel-book_people','hotel-book_stay'), ('restaurant-book_people','hotel-book_stay')]

FROM_SCRATCH = True

MODEL_PATH = './model.pt'

TRIPPY_CONFIG = TRIPPYConfig(multiwoz=True)

BERT_CONFIG = BERTConfig()

# Build model & Load weights

In [177]:
tods = DeepTODS(CHATBOT_NAME, DOMAINS, ONTOLOGY_PATH, LABEL_MAPS_PATH, POLICY, START, TEMPLATES, NON_REFERABLE_SLOTS, NON_REFERABLE_PAIRS, FROM_SCRATCH, BERT_CONFIG, TRIPPY_CONFIG)
tods.load_dst_model(MODEL_PATH)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded successfully.


# Conversation

In [178]:
# chat_gui = chat_gui(tods.infer)
# chat_gui.run()

In [179]:
tods.reset()
print(tods.get_dialogue_state())

{}


In [180]:
response = tods.infer('')
print(tods.get_dialogue_state())
print(response)

{}
Hi I am Tody, I can help you reserve a hotel or a taxi?


In [181]:
response = tods.infer('Hi Tody, I want to book a hotel in the city center.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center'}
Do you have a price reference?


In [182]:
response = tods.infer('yes I want a moderate price range.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate'}
What day would you like to stay and how many days?


In [183]:
response = tods.infer('I want to check in on monday and I am staying for 2 days.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate', 'hotel-book_day': 'monday', 'hotel-book_stay': '2'}
I have one called the continental hotel.


In [184]:
response = tods.infer('Ok book it please.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate', 'hotel-book_day': 'monday', 'hotel-book_stay': '2', 'hotel-name': '§§continental hotel'}
Do you need parking or internet?


In [185]:
response = tods.infer('Yes I need parking and internet.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate', 'hotel-book_day': 'monday', 'hotel-book_stay': '2', 'hotel-name': '§§continental hotel', 'hotel-internet': 'yes', 'hotel-parking': 'yes'}
How many people is the booking for?


In [186]:
response = tods.infer('I am booking for 2 people.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate', 'hotel-book_day': 'monday', 'hotel-book_stay': '2', 'hotel-name': '§§continental hotel', 'hotel-internet': 'yes', 'hotel-parking': 'yes', 'hotel-book_people': '2'}
What is your desired destination?


In [187]:
response = tods.infer('I want to book a taxi to go to the hotel.')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate', 'hotel-book_day': 'monday', 'hotel-book_stay': '2', 'hotel-name': '§§continental hotel', 'hotel-internet': 'yes', 'hotel-parking': 'yes', 'hotel-book_people': '2', 'taxi-destination': '§§continental hotel'}
From where you will be departing, and what time do you want to leave?


In [188]:
response = tods.infer('I am leavivng from the ramses station at 10 am')
print(tods.get_dialogue_state())
print(response)

{'hotel-area': 'center', 'hotel-pricerange': 'moderate', 'hotel-book_day': 'monday', 'hotel-book_stay': '2', 'hotel-name': '§§continental hotel', 'hotel-internet': 'yes', 'hotel-parking': 'yes', 'hotel-book_people': '2', 'taxi-destination': '§§continental hotel', 'taxi-departure': 'ramses station', 'taxi-leaveAt': '10:00'}
None
